In [ ]:
!pip install fasttext
import fasttext
import fasttext.util
# ft_path = '/home/zofia/my_files/studia/boty/data/fasttext/cc.pl.100.bin'
ft_path = '../data/fasttext/cc.pl.100.bin'

In [ ]:
ft.get_word_vector('domek')

NameError: ignored

In [ ]:
# clarin_path = '/home/zofia/my_files/studia/boty/data/clarin/skipgram/skip_gram_v100m8.w2v.txt'
# import numpy as np
# word2vector = {}
# with open(clarin_path, 'r') as cl:
#     cl.readline()
#     for line in cl:
#         if line.strip():
#             word = line.split()[0].split('::')[0]
#             vector = line.split()[1:]
#             vector = np.array(vector)
#             word2vector[word] = vector

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
main_path = 'gdrive/MyDrive/bot/data'
# subtitles_part_path = f"{main_path}/open_subtitles/xaa"
# vectors_path = f"{main_path}/xaa_vectors.json"
import re
import numpy as np
import json
import os

ft_path = os.path.join(main_path, 'fasttext/cc.pl.100.bin')
vectorss_path = os.path.join(main_path, 'open_subtitles/1MB_vectors')

ft = fasttext.load_model(ft_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
word_vectors_dict = {}
def get_sentence_vector(sentence):
    words = sentence.split()
    # leave only letters and numbers
    words = [re.sub(r'[\W_]+', '', word) for word in words]
    
    word_vectors = []
    for word in words:
        if word not in word_vectors_dict:
            word_vectors_dict[word] = ft.get_word_vector(word)
        word_vectors.append(word_vectors_dict[word])

    word_vectors = list(map(np.array, word_vectors))
    vectors_sum = np.add.reduce(word_vectors)
    vectors_mean = vectors_sum / len(words)
    return vectors_mean

In [ ]:
#decode compressed vectors
import re

def decode_compressed_vector(compressed):
  compressed = compressed[1:-2]
  partly_decompressed = re.findall('......',compressed)
  partly_decompressed_with_dots = [num[:2] + "." + num[2:] for num in partly_decompressed]
  return list(map((lambda num: float(num[1:]) if num.startswith("+") else float(num)), partly_decompressed_with_dots))

In [ ]:

import numpy as np
from array import array

subtitles_part_path = f"{main_path}/tenthousand_corpus.txt"
vectors_path = f"{main_path}/corpus_vectori.csv"
def make_sentence_vectors_file():
    with open(subtitles_part_path, 'r', encoding="utf-8") as subtitles:
        with open(vectors_path, 'w') as vector_file:
          dict_to_df = {}
          for line in subtitles:
              dict_to_df[line] = get_sentence_vector(line)
          df = pd.DataFrame(data=dict_to_df)





# for line in file.read():
#    word, data = line.split(' ')
#    vector = numpy.array(data, datatype=float16)

#             df = pd.DataFrame(data=d)
#             for i, line in enumerate(subtitles):
#                 line = line.strip()
#                 if line:
#                     if i % 10000 == 0:
#                         print(i)
#                     sent_vector = get_sentence_vector(line)
#                     #sent_vector = [line] + sent_vector
#                     arr = array('d', sent_vector)
#                     print(line)
#                     print(sent_vector)
#                     print(arr)
#                     arr.tofile(vector_file)
#                     #vector_file.write("\n")
#                     input()

make_sentence_vectors_file()

UnicodeEncodeError: ignored

In [ ]:
# vectors_path = f"{main_path}/xaa10MB_vectors_fasstext_dim100.json"
# subtitles_part_path = f"{main_path}/open_subtitles/xaa"



######################################
#######################################
# Mocniejsza kompresja jest możliwa - trzeba wrzucać rzeczy do jednego słownika na nie słownik, uciąć nawiasy w wektorach bo po co"[aaabbbccc]"
# być może przechowywać jakoś inaczej -string-> do pliku
#czytanko o kompresji danych
def my_round_to_string(num):
  string_number = str(round(num, 4)).replace(".", "")
  if len(string_number) < 6:
    if not string_number.startswith("-"):
        string_number = "+" + string_number
    for i in range(6 - len(string_number)):
      string_number += "0"
  return string_number.replace(".", "")
###
"""
dla c++:
float16_t x = 42.42f;
char* float_do_pliku = (char*) &x;

cout << float_do_pliku[0] <<
float_do_pliku[1] << " ";

chcemy wziąć pamięć w której zapisany jest float
powiedzieć pythonowi, żeby potraktował ten adres w pamięci jako raw stringa
i go zapisał do pliku
i w ten sposób on sobie wpisze bezsenowne znaczki, które nie będą w ogóle czytelne
a potem wczytam se po 2 znaczki i teraz te 2 bajty charów będę traktować jako floata16
i dostanę to co podałam

"""

for line in file.read():
   word, data = line.split(' ')
   vector = numpy.array(data, datatype=float16)

subtitles_part_path = f"{main_path}/pl_tok_sieved.txt"
vectors_path = f"{main_path}/corpus_vectors_dim100.json"
def make_sentence_vectors_file():
    with open(subtitles_part_path, 'r', encoding="utf-8") as subtitles:
        with open(vectors_path, 'w', encoding="utf-8") as vector_file:
            for i, line in enumerate(subtitles):
                line = line.strip()
                if line:
                    if i % 10000 == 0:
                        print(i)
                    sent_vector = get_sentence_vector(line)
                    sent_vector = sent_vector.tolist()
                    sent_vector = [round(num, 4) for num in sent_vector]
                    sent_vector = [my_round_to_string(num) for num in sent_vector]
                    list_without_spaces = str(sent_vector).replace(" ", "")    
                    list_without_quotes_marks = list_without_spaces.replace("'", "")
                    list_without_separators = list_without_quotes_marks.replace(",", "")       
                    to_write = json.dumps({line: list_without_separators}, ensure_ascii=False) #either we add ensure_ascii = False or change the encoding when reading from a file.
                    if i < 10 :
                        print(line)
                        print(to_write)
                    if i == 3000000:
                      break
                    vector_file.write(f"{to_write}\n")


make_sentence_vectors_file()

0
Wykrwawi się na śmierć !
{"Wykrwawi się na śmierć !": "[-00588-00980+00658-00092-01059+00061+00485-02466+01470+00881-00003-00375+00271-00490-00176+00563-00585+00185+00151+00724-00565+00072+00085-00170+00671-00418-01200-00034+00401+00312-00441-00603+00002+00292-00493-00029+01095-00190+00729-00142+00961+00136-00250+00464+00158-00139-00574+00060+00357-00081-01031-01316-01837-00601+00016-00063-00418+00074-00039+00233+00169+00011+00534+00363-00294+00367+00210-00264+00137-00423+00236-00482+00301-00195-00195-00446-00054+00258-00199-00367-00132-00295+00589+00234-00028+00068-00619-00074-00018+00562+00318+00133-00098+00298-00516-00198-00531-00453+00948+00210]"}
Są za nami ?
{"Są za nami ?": "[+01055-01595+02356-01461-00317+00037+01282-03948+01398+03587-01793-00368-00810-00474+00739-01030-00036+01308-00813-00038-00856+00296+01078+00670-00017-00967-02023+00846-00627+00576-00613-00056-00113-00081-00078-00810+02607+00816+00484-00892-00726-00719-00252+01078-00106-00187+00445-00457+00446+00001-01326

In [ ]:
vectors_path = f"{main_path}/1MB_vectors/"
subtitles_part_path = f"{main_path}/open_subtitles/1MB/"

from os import listdir
from os.path import isfile, join

def make_sentence_vectors_file(path_in, path_out):

    with open(path_in, 'r', encoding='utf8') as subtitles:
        with open(path_out, 'w', encoding='utf8') as vector_file:

            for i, line in enumerate(subtitles):
                line = line.strip()
                if line:
                    if i % 10000 == 0:
                        print(i)
                    # if i > 10000:
                    #     break
                    sent_vector = get_sentence_vector(line)
                    sent_vector = sent_vector.tolist()
                    sent_vector = [round(num, 4) for num in sent_vector]
                    to_write = json.dumps({line: sent_vector})
                    if i <2 :
                        print(to_write)
                    vector_file.write(f"{to_write}\n")
    
def make_ten_files():
    files = listdir(subtitles_part_path)
    for p in files:
        print(p)
        path_in = join(subtitles_part_path, p)
        path_out = join(vectors_path, p)
        make_sentence_vectors_file(path_in, path_out)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from operator import itemgetter

In [ ]:
def find_cosine_similarity(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [ ]:
#sorting vectors by cosine similarity

from math import sqrt

def cosine_distance(v):
    return sqrt(sum(x**2 for x in v))

vectors_path = f"{main_path}/corpus_vectors_dim100.json"
list_of_vectors = []
with open(vectors_path, "r") as vectors:
  #tu trzeba przygotować listę wektorów
  for line in vectors:
    line = json.load(line)
    vector = 



sorted(myListOfVectors, key=euclidean_distance)

In [ ]:
def similarity_by_embeddings(sentence_in, how_many, vector_file):
    vector_in = get_sentence_vector(sentence_in)
    sentence2similarity = {}
    with open(vector_file, 'r', encoding='utf8') as vectors:
        for line in vectors:
            line = json.loads(line)

            # theres only one pair (key-value) in every line
    
            sentence_out, vector_out = list(line.items())[0]
            similarity = find_cosine_similarity(vector_in, vector_out)
        
            if similarity > 0.5:
                sentence2similarity[sentence_out] = similarity
    res = dict(sorted(sentence2similarity.items(), key = itemgetter(1), reverse = True)[:how_many]) 
    return res

In [ ]:
import random

def choose_random():
    files = listdir(vectors_path)
    random_file = random.choice(files)
    return join(vectors_path, random_file)

def find_sentence(sentence_in, how_many):
    vector_file = choose_random()
    print(vector_file)
    res = similarity_by_embeddings(sentence_in, how_many, vector_file)
    res_sentence = list(res.keys())[0]
    return res_sentence

In [ ]:
print(find_sentence("Ten za łukiem rzeki świat.", 1))

/home/zofia/my_files/studia/boty/data/1MB_vectors/xai
Pod słonecznym niebem Co za słodki los


In [ ]:
find_sentence("Czy możesz mi pomóc?.", 1)

'Nie możesz zabrać mi telewizora.'

In [ ]:
def talk():
    print("Cześć! Jestem bot Helfik :) W czym mogę Ci pomóc?")
    sentence_in = input()
    while(sentence_in != "quit"):
        sentence_type = find_sentence_type(sentence_in)
        
        sentence_out = find_sentence(sentence_in, 1)
        print(sentence_out)
        sentence_in = input()   

In [ ]:
talk()

Cześć! Jestem bot Helfik :) W czym mogę Ci pomóc?
/home/zofia/my_files/studia/boty/data/1MB_vectors/xaj
- Czy była ładna?
/home/zofia/my_files/studia/boty/data/1MB_vectors/xad
Pewnie tak.


In [ ]:
recipe_path = "/home/zofia/my_files/studia/boty/data/przepisy/recipes_raw_przepisy.json"
# recipe_path = recipe_cleared_path
recipe_vectors_path = "/home/zofia/my_files/studia/boty/data/przepisy/recipes_vectors.json"

import json
def make_ingredients_vector_file():
    with open(recipe_path, 'r') as recipe_file:
        with open(recipe_vectors_path, 'w') as vectors_file:
            recipes = json.load(recipe_file)
            for url, recipe in recipes.items():
                try:
                    ingredients = recipe["ingredients"]
                except KeyError:
                    print(recipe)
                ingredients_vector = get_sentence_vector(' '.join(ingredients))

                ingredients_vector = ingredients_vector.tolist()
                ingredients_vector = [round(num, 4) for num in ingredients_vector]
                to_write = json.dumps({url: ingredients_vector})
                vectors_file.write(f"{to_write}\n")

In [ ]:
def get_urls2(ingredients, how_many):
    ingredients_list = ingredients.split()
    ingredients_list = [ingredient.strip(',.') for ingredient in ingredients_list]
    ingredients_vectors = list(map(get_sentence_vector, ingredients_list))
    res_urls = []
    with open(recipe_vectors_path, 'r', encoding='utf8') as vectors:
        for line in vectors:
            line = json.loads(line)

            # theres only one pair (key-value) in every line
            url_out, vector_out = list(line.items())[0]
            similarities = [find_cosine_similarity(vector_out, vector_in) for vector_in in ingredients_vectors]

            if all(similarity > 0.7 for similarity in similarities):
                res_urls.append(url_out)
    return res_urls

In [ ]:
def get_urls1(ingredients, how_many):
    vector_in = get_sentence_vector(ingredients)
   
    url2similarity = {}
    with open(recipe_vectors_path, 'r', encoding='utf8') as vectors:
        for line in vectors:
            line = json.loads(line)

            # theres only one pair (key-value) in every line
            url_out, vector_out = list(line.items())[0]
        
            similarity = find_cosine_similarity(vector_in, vector_out)
            
            if similarity > 0.6:
                url2similarity[url_out] = similarity
    res = dict(sorted(url2similarity.items(), key = itemgetter(1), reverse = True)[:how_many]) 
    return res

In [ ]:
recipe_path = "/home/zofia/my_files/studia/boty/data/przepisy/recipes_raw_przepisy.json"
recipe_cleared_path = "/home/zofia/my_files/studia/boty/data/przepisy/recipes_cleared.json"
def clear_recipes():
    cleared_recipes_dict = {}
    with open(recipe_path, 'r') as recipe_file:
        recipes = json.load(recipe_file)
        for url, recipe in recipes.items():
            try:
                ingredients = recipe["ingredients"]
                instructions = recipe['instructions']
                title = recipe['title']
            except KeyError:
                continue
            if len(instructions) < 50:
                continue
            if recipe['title'] != 'bleblelbel' and recipe['title'] != 'Shake czekoladowy' and recipe['title'] != 'Zapiekane ziemnaki z serem i masłem':
                cleared_recipes_dict[url] = recipe 
    with open(recipe_cleared_path, 'w') as cleared_file:
        json.dump(cleared_recipes_dict, cleared_file)

In [ ]:
clear_recipes()

In [ ]:
recipe_path = recipe_cleared_path

In [ ]:
def find_recipes2(ingredients_in):
    ingredients_in = ingredients_in.split()
    res = []
    with open(recipe_path, 'r') as recipe_file:
        recipes = json.load(recipe_file)
        for url, recipe in recipes.items():
            try:
                ingredients_out = recipe["ingredients"]
            except KeyError:
                pass
            pref = 3
            ingredients_out_prefs = [ingredient[:pref] for ingredient in ingredients_out]
            ingredients_in_prefs = [ingredient[:pref] for ingredient in ingredients_in]
            if ingredients_out and all(ingredient[:pref] in ingredients_in_prefs for ingredient in ingredients_out):
                res.append(recipe)
    return res

In [ ]:
find_recipes2('mleko jajko oliwki ser mąka')

[{'title': 'Zupa mleczna z zacierkami',
  'ingredients': ['jajko 1 sztuka', 'mąka pszenna 4 łyżki', 'mleko 1 litr'],
  'instructions': 'Z jajka i mąki przygotuj twarde ciasto i dobrze je wyrób.\nUtrzyj je na tarce lub posiekaj drobno. Zacierki wrzuć do wrzącej wody i gotuj około 8 minut.\nPo ugotowaniu odcedź na sitku i dodaj do świeżo przegotowanego mleka.',
  'picture_link': None},
 {'title': 'Zupa mleczna z lanymi kluskami',
  'ingredients': ['mąka pszenna 2 łyżki', 'mleko 1 litr', 'jajka 2 sztuki'],
  'instructions': 'Zagotuj mleko.\nPrzygotuj ciasto: przesiej mąkę, dodaj do rozmąconego jajka i ubij na jednolitą masę.\nCiasto cienkim strumieniem wlewaj do gotującego się mleka.',
  'picture_link': None},
 {'title': 'Babeczki kakaowe',
  'ingredients': ['mąka 250 gramów', 'jajka 2 sztuki'],
  'instructions': 'Do miski przesiewamy suche składniki czyli mąkę i dodajemy cukier, proszek do pieczenia, cukier wanilinowy, ciemne kakao oraz rodzynki.\nDo drugiej miski dodajemy składniki mokr

In [ ]:
def find_recipes(ingredients, how_many):
    urls = get_urls2(ingredients, how_many)
    recipes_out = []
    with open(recipe_path, 'r') as recipe_file:
        recipes = json.load(recipe_file)
        for url_in, recipe in recipes.items():
            if url_in in urls:
                recipes_out.append(recipe)
    return recipes_out

In [ ]:
def get_similarity(sentence_A, sentence_B):
    vector_A = get_sentence_vector(sentence_A)
    vector_B = get_sentence_vector(sentence_B)
    similarity = find_cosine_similarity(vector_A, vector_B)
    return similarity

In [ ]:
get_similarity("jajko", 'jajka mleko woda mąka')

0.7309807

In [ ]:
find_recipes2("oliwki ser żółty jogurt naturalny sałata jajko mąka")

[{'title': 'Babeczki kakaowe',
  'ingredients': ['mąka 250 gramów', 'jajka 2 sztuki'],
  'instructions': 'Do miski przesiewamy suche składniki czyli mąkę i dodajemy cukier, proszek do pieczenia, cukier wanilinowy, ciemne kakao oraz rodzynki.\nDo drugiej miski dodajemy składniki mokre czyli wbijamy jajka, wlewamy mleko oraz olej rzepakowy. Dokładnie mieszamy trzepaczką, jak składniki mokre się połączą dodajemy do suchych składników. Wszystko dokładnie mieszamy, a powstałą masą napełniamy silikonowe foremki.\nSilikonowe foremki wstawiamy do piekarnika nagrzanego do 200 stopni C na około 25 minut (albo termoobieg 180). Po ostygnięciu posypujemy babeczki kakaowe cukrem pudrem.',
  'picture_link': None}]

In [ ]:
len('wszystko zagniesc\ndo pieca\nostudzic w lodowce')

45